In [118]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
path = 'C:/Users/ah3wj/Desktop/epi-segmentation/'
df = pd.read_csv(path + 'covid-19-dataset/us-counties.csv')

In [3]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [4]:
df.county.head()

0    Snohomish
1    Snohomish
2    Snohomish
3         Cook
4    Snohomish
Name: county, dtype: object

In [5]:
df['date'] = df['date'].astype('datetime64[ns]')

In [6]:
df.sort_values(by = ['date'], inplace = True, ascending = True)

In [7]:
all_dates = df.date.unique()
print(len(all_dates))

94


In [32]:
df_counties = df['county']  + ':' +df['state']
all_counties = df_counties.unique()
print(len(all_counties))

2836


In [50]:
def preprocess(lookup, resolution):
  with open(path + 'covid-19-dataset/processed-data/processed-us-counties-' + lookup + '.csv', 'w+') as lookup_file:

    with open(path + 'covid-19-dataset/processed-data/us-counties-order.csv', 'w+')  as order_file:

      for x in all_counties:
        county, state = x.split(':') 
#         print(county + '\n', end='')
        order_file.write(county + '\n')

        for date in all_dates:
          filter1 = df[resolution] == county
          filter2 = df['state'] == state
          filter3 =  df['date'] == date

          if((filter1 & filter2 & filter3).any()):
            value = df.loc[filter1 & filter2 & filter3][lookup].to_string(index=False)
#             if(not value.isnumeric()):
#               value = '0'
          else:
            value = '0'

          if(date == all_dates[-1]):
#             print(value , end='')
            lookup_file.write(value)
          else:
#             print(value + ', ', end='')
            lookup_file.write(value + ', ')

#         print('\n')
        lookup_file.write('\n')

In [36]:
########################### careful here 
########################################
########################################

lookup = 'cases'
resolution = 'county'

preprocess(lookup= lookup, resolution = resolution)


In [51]:
with open(path + 'covid-19-dataset/processed-data/us-counties-order.csv', 'w+')  as order_file:
  for x in all_counties:
    order_file.write(x + '\n')        


**Now Time to Verify**

In [37]:
filename = path + 'covid-19-dataset/processed-data/processed-us-counties-' + lookup + '.csv'
verification = np.loadtxt(filename, dtype=int, delimiter=',')

print(verification)

[[    1     1     1 ...  2162  2208  2239]
 [    0     0     0 ... 23181 24546 25811]
 [    0     0     0 ...  1691  1753  1827]
 ...
 [    0     0     0 ...     0     0     1]
 [    0     0     0 ...     0     0     1]
 [    0     0     0 ...     0     0     1]]


In [25]:
# flag = False

# for i in range(len(verification)):

#   if(flag):
#     break
    
#   county = all_counties[i]   
# #   print(state)
#   for j in range(len(verification[i])):
#     if(verification[i][j] != 0):
#       date = all_dates[j]

# #       print(date)
      
#       df_of_county = df[df[resolution]== county]
#       df_of_time = df_of_county[df_of_county['date'] == date]
#       value = int(df_of_time[lookup].to_string(index=False))
      
#       assert value == verification[i][j]
#       if( value != verification[i][j]):
#         print("Doesn't match", state, date)
#         print('verif index: ', j, 'value: ', verification[i][j])
#         print('actual: ', value)
#         flag = True
#         break
        

In [52]:
counties_order = pd.read_csv(path+'covid-19-dataset/processed-data/us-counties-order.csv', header=None, encoding = "ISO-8859-1")

In [53]:
data = pd.read_csv(path+'covid-19-dataset/processed-data/processed-us-counties-cases.csv', header=None)

In [54]:
counties_order.head()

,0
0,Snohomish:Washington
1,Cook:Illinois
2,Orange:California
3,Maricopa:Arizona
4,Los Angeles:California


In [55]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,1,1,1,1,1,1,1,1,1,1,...,1949,1987,2031,2055,2092,2142,2162,2162,2208,2239
1,0,0,0,1,1,1,1,1,1,2,...,16323,17306,18087,19391,20395,21272,22101,23181,24546,25811
2,0,0,0,0,1,1,1,1,1,1,...,1299,1376,1425,1501,1556,1636,1676,1691,1753,1827
3,0,0,0,0,0,1,1,1,1,1,...,2056,2146,2264,2404,2491,2589,2636,2738,2846,2970
4,0,0,0,0,0,1,1,1,1,1,...,10047,10496,10854,11391,12021,12341,13816,15140,16435,17508


In [56]:
frames = [counties_order, data]

In [57]:
result = pd.concat(frames, axis = 1)

In [58]:
result.head()

,0,0,1,2,3,4,5,6,7,8,...,84,85,86,87,88,89,90,91,92,93
0,Snohomish:Washington,1,1,1,1,1,1,1,1,1,...,1949,1987,2031,2055,2092,2142,2162,2162,2208,2239
1,Cook:Illinois,0,0,0,1,1,1,1,1,1,...,16323,17306,18087,19391,20395,21272,22101,23181,24546,25811
2,Orange:California,0,0,0,0,1,1,1,1,1,...,1299,1376,1425,1501,1556,1636,1676,1691,1753,1827
3,Maricopa:Arizona,0,0,0,0,0,1,1,1,1,...,2056,2146,2264,2404,2491,2589,2636,2738,2846,2970
4,Los Angeles:California,0,0,0,0,0,1,1,1,1,...,10047,10496,10854,11391,12021,12341,13816,15140,16435,17508


In [59]:
correct_format = result.T

In [60]:
correct_format.to_csv(path+'covid-19-dataset/processed-data/correct-state-cases.csv', header = False, index = False)

In [73]:
county_id = pd.read_csv(path+'covid-19-dataset/nssac_county_data.csv')

In [74]:
county_id.head()

,FIPS,County,Province_State,Country_Region,Last_Updated,Latitude,Longitude,Confirmed,Deaths,Recovered,Active
0,1001.0,Autauga,Alabama,USA,2020-05-08 00:00:00,32.534920,-86.642749,61,3,0,58
1,1003.0,Baldwin,Alabama,USA,2020-05-08 00:00:00,30.660970,-87.749840,205,5,0,200
2,1005.0,Barbour,Alabama,USA,2020-05-08 00:00:00,31.869603,-85.393197,51,1,0,50
3,1007.0,Bibb,Alabama,USA,2020-05-08 00:00:00,32.998644,-87.126439,44,0,0,44
4,1009.0,Blount,Alabama,USA,2020-05-08 00:00:00,33.980867,-86.567371,44,0,0,44


In [80]:
county_state = county_id[' County']+':'+county_id[' Province_State']
county_id['county_state'] = county_id[' County']+':'+county_id[' Province_State']

In [82]:
county_id = county_id[['FIPS', 'county_state']]

In [94]:
county_id = county_id.dropna()

In [99]:
county_id

,FIPS,county_state
0,1001,Autauga:Alabama
1,1003,Baldwin:Alabama
2,1005,Barbour:Alabama
3,1007,Bibb:Alabama
4,1009,Blount:Alabama
...,...,...
2951,56035,Sublette:Wyoming
2952,56037,Sweetwater:Wyoming
2953,56039,Teton:Wyoming
2954,56041,Uinta:Wyoming


In [98]:
county_id['FIPS'] = county_id['FIPS'].astype('int32')

C:\Users\ah3wj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [114]:
adj = pd.read_csv(path+'covid-19-dataset/county_neighbors.csv')  

In [115]:
adj = adj[['Source', 'Destination']]

In [116]:
adj = adj.to_csv(path+'covid-19-dataset/correct_county_edge_id_list.csv', header = False, index=False)

*Now networkx